<a href="https://colab.research.google.com/github/zosimanoz/CotivitiHackathon-6.0/blob/main/NER_hackathon_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("drive/My Drive/HackaFest")

In [ ]:
!pip install spacy[transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 26 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
import json

with open('PHI_dataset_train_bak.json', 'r') as f:
    train_data = json.load(f)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_trf')

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

# nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(train_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        # print(start, end, label)
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

 25%|██▍       | 631/2539 [00:00<00:01, 1306.65it/s]

Skipping entity


100%|██████████| 2539/2539 [00:01<00:00, 1340.79it/s]


In [ ]:
import json

with open('PHI_dataset_test.json', 'r') as f:
    test_data = json.load(f)

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

db = DocBin() # create a DocBin object

for text, annot in tqdm(test_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        # print(start, end, label)
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./dev.spacy") # save the docbin object

100%|██████████| 440/440 [00:00<00:00, 1566.94it/s]


In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./checkpoint_trf --paths.train ./train.spacy --paths.dev ./dev.spacy --gpu-id 0

✔ Created output directory: checkpoint_trf
ℹ Saving to output directory: checkpoint_trf
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-11-25 18:27:18,235] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-11-25 18:27:18,246] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2022-11-25 18:27:18,250] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-11-25 18:27:18,252] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertFor

In [ ]:
nlp1 = spacy.load("./checkpoint_trf/model-last") #load the best model
doc = nlp1("Manoj Pradhan with plan H2214 on 21061 2014-03-06 F 7140 2004-03-15 went to hospital radiology 310-39-2808 +1 8411155783 Kathmandu Nepal") # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter